In [51]:
# dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [52]:
# read cancercsv into dataframe
cancer_df = pd.read_csv("static/csv/skin_map_incidence.csv")
cancer_df.head()

,State,Range,Rate
0,AK,9.5 to 19.0,14.9
1,AL,9.5 to 19.0,19.0
2,AR,19.1 to 21.9,20.0
3,AZ,19.1 to 21.9,20.6
4,CA,23.0 to 25.8,23.0


In [53]:
# read solar install csv into dataframe
solar_df = pd.read_csv("static/csv/openpv-export-201803141836.csv")
solar_df.head(2)

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,effic_1,cust_county,tracking,inv_model_clean,mod_cost_real,inv_cost_real,bos_powerclerk_real,permitting_real,3rdparty,Filters=maxdate=1420045199&mindate=1388552400
0,CO,01/01/2014,NaN,uir,13.58,NaN,80401,Government,NaN,1.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NY,01/01/2014,NaN,uir,306.60,NaN,11215,commercial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
solar_targs_df = solar_df.iloc[:,[0, 1, 4, 6, 9]]
solar_targs_df.columns = ["state", "install_Data", "size_kw", "zip", "cost_per_Watt"]
solar_targs_df.head(2)

,state,install_Data,size_kw,zip,cost_per_Watt
0,CO,01/01/2014,13.58,80401,1.62
1,NY,01/01/2014,306.60,11215,NaN


In [55]:
# check to see how deeply the nans permiate our data
solar_targs_df.count()


state            170862
install_Data     170862
size_kw          170862
zip              170862
cost_per_Watt     73212
dtype: int64

In [56]:
# read eia cost data into dataframe
eia_df = pd.read_csv("static/csv/eia_2014_scrape.csv")
eia_df = eia_df.iloc[:,[1,2]]
eia_df["State"] = ["AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA","KS",
                   "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC",
                   "ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
#eia_df

In [57]:
# merge cancer and electric cost dfs
cande_df = pd.merge(cancer_df, eia_df, on="State", how="outer")
cande_df.columns = ["state", "range", "rate", "name", "average retail price (cents/kWh)"]
electric_cancer = cande_df[["name","state","rate","average retail price (cents/kWh)"]]
electric_cancer.columns = ["state","sbbr","cancer_Rate","electric_Cost"]
#electric_cancer.head()
electric_cancer.to_csv("static/csv/electric_cost_cancer_rate.csv")

In [58]:
# merge solar and cande dfs
data_df = pd.merge(cande_df, solar_targs_df, on="state", how="outer")

# reset index to serve as id column
data_df = data_df.reset_index()
#data_df.head()

# rename columns and reorder columns
data_df.columns = ["id", "abbr", "c_range", "cancer_Rate", "state", "electric_Cost_cents_kWh)", "install_Date",
                  "size_kW", "zipcode", "cost_per_Watt"]
data_df = data_df[["id", "state", "abbr", "electric_Cost_cents_kWh)", "install_Date", "zipcode", "size_kW",
                 "cost_per_Watt", "c_range", "cancer_Rate"]]
data_df["size_kW"].fillna(0, inplace=True)
#data_df.head(10)

# push clean df to csv
data_df.to_csv("static/csv/clean_data.csv")

In [59]:
one = []

for i in data_df["size_kW"]:
    i == data_df["size_kW"]
    if 0 < i < 10:
        one.append(i)
    else :
        one.append(0)

two = []
for i in data_df["size_kW"]:
    i == data_df["size_kW"]
    if 10 <= i < 100:
        two.append(i)
    else :
        two.append(0)

three = []
for i in data_df["size_kW"]:
    i == data_df["size_kW"]
    if i >= 100:
        three.append(i)
    else :
        three.append(0)

data_df["size_one"] = one
data_df["size_two"] = two
data_df["size_three"] = three

#data_df.head(50)

In [60]:
# create df with total installed kW by state
installed = data_df.groupby(["abbr"])["size_kW", "size_one", "size_two", "size_three"].sum()
installed_df = pd.DataFrame(installed)
installs_df = installed_df.reset_index()
#installs_df
installs_df.to_csv("static/csv/intall_totals_state.csv")

In [61]:
data_df.drop("size_one", axis=1, inplace=True)
data_df.drop("size_two", axis=1, inplace=True)
data_df.drop("size_three", axis=1, inplace=True)
#data_df.head()

In [67]:
# push data to sqlite db
engine = create_engine('sqlite:///clean_solar_data.db')
data_df.to_sql("data", con=engine, flavor="sqlite", if_exists="replace", index=False)
installs_df.to_sql("install_total", con=engine, flavor="sqlite", if_exists="replace", index=False)
electric_cancer.to_sql("electric_cancer", con=engine, flavor="sqlite", if_exists="replace", index=False)

C:\Users\brbal\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\pandas\io\sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


In [70]:
# test db completion
df = pd.read_sql_query("SELECT * FROM 'install_total' LIMIT 10", engine)
df.head()

,abbr,size_kW,size_one,size_two,size_three
0,AK,1.450000e+01,2.500000,12.000000,0.000000
1,AL,0.000000e+00,0.000000,0.000000,0.000000
2,AR,0.000000e+00,0.000000,0.000000,0.000000
3,AZ,1.551172e+05,73347.394333,43593.181000,38176.638000
4,CA,1.173274e+06,510726.487028,124146.322401,538401.221577
